In [96]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
df = pd.read_csv("https://raw.githubusercontent.com/gmfv/YIO/main/datosfiltrados.csv",index_col=0,skiprows=0)

 **Eliminamos algunas columnas**

In [ ]:
copy_df=df.copy()
#df = df.drop(df.columns[[0, 1, 2, 3, 5, 7, 12]], axis='columns')
#df.head()
copy_df=copy_df.set_index("id_anony")
copy_df=copy_df[copy_df["Asignatura"] == "CALCULO 2"]
columns_keep=["Cod.Car.Sec","Anho","Semestre", "Aprobado", "Anho.Firma",
              "Primer.Par", "Segundo.Par", "AOT"]
copy_df=copy_df[columns_keep]
copy_df#.head()

**Reemplazamos las salidas por 1 o 0, las carreras por codigos**

In [100]:
copy_df['Aprobado']=copy_df['Aprobado'].replace(['N', 'S'],['0', '1'])
carreras=pd.Series(['0', '1', '2', '3', '4', '5', '6', '7', '8','9'],index=[copy_df['Cod.Car.Sec'].unique()])
#copy_df['Cod.Car.Sec']=copy_df['Cod.Car.Sec'].replace(copy_df['Cod.Car.Sec'].unique(),['0', '1', '2', '3', '4', '5', '6', '7', '8','9'])
copy_df['Cod.Car.Sec']=copy_df['Cod.Car.Sec'].replace(carreras)
#copy_df = copy_df.fillna(0)
copy_df#.head()

,Cod.Car.Sec,Anho,Semestre,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT
id_anony,,,,,,,,
es_1444,0,2017,1,0,0,7,0,0
es_4238,1,2017,1,0,0,5,0,0
es_4245,0,2017,1,1,2016,0,0,0
es_4967,2,2017,1,0,0,0,0,0
es_414,1,2017,1,1,2017,21,10,0
...,...,...,...,...,...,...,...,...
es_9590,5,2019,2,0,0,5,4,2
es_9697,2,2019,2,0,0,5,0,5
es_9437,2,2019,2,0,0,0,0,0


In [ ]:
df[df.AnhoFirma == df.Anho]

**Descartamos columnas para facilitar analisis inicial**

---
Solo tenemos en cuenta el rendimiento de primer, segundo parcial y carrera


In [120]:
X = np.array(copy_df.drop(['Cod.Car.Sec', 'Anho','Semestre','Anho.Firma','Aprobado'], axis=1))
y = np.array(copy_df['Aprobado'])

array([[ 7,  0,  0],
       [ 5,  0,  0],
       [ 0,  0,  0],
       ...,
       [ 0,  0,  0],
       [11, 14, 10],
       [ 0,  0,  0]])

In [102]:
model = linear_model.LogisticRegression()
model.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [103]:
predictions = model.predict(X)
print(predictions)

['0' '0' '0' ... '0' '0' '0']


Precision media de:

In [104]:
model.score(X,y)

0.8090737240075614

# **VALIDACION**

Dividimos nuestros datos en una seccion para entrenar y otra para validar

In [112]:
validation_size = 0.20
seed = 23
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, y, test_size=validation_size, random_state=seed)

Entrenamos a nuestro modelo con el 80% de los datos. Obteniendo una media de exactitud y desviacion estandar que se imprimen en pantalla

In [113]:
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(msg)

Logistic Regression: 0.814817 (0.032689)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [114]:
predictions = model.predict(X_validation)
print(accuracy_score(Y_validation, predictions))

0.8050314465408805


Tener en cuenta que la cantidad de datos reservados para la validacion era pequena en comparacion

# **Matrix de confusion**

In [118]:
print(confusion_matrix(Y_validation, predictions))

[[118  32]
 [ 30 138]]


32 que aprobaron fueron predecidos como no aprobados

30 que no aprobaron fueron predecidos como aprobados

# **Prueba** 

In [117]:
X_new = pd.DataFrame({'Primer.Par': [0], 'Segundo.Par': [24], 'AOT': [4]})
model.predict(X_new)

array(['0'], dtype=object)

# Nueva sección